In [1]:
import os
import pandas as pd

dataset_path = os.path.realpath(os.path.join(os.path.dirname(__name__), '..', 'app','static','uploads', 'dataset.csv'))
df_comments = pd.read_csv(dataset_path)
df_comments.tail(n=99)

,comment,label
0,Sumpah ya Avoskin cica mugwort gue pKe malah j...,negatif
1,Selama ini base makeupnya wardah kayak susah b...,negatif
2,Ka Lip itu lipstik yg dipegang waktu akhir2 pa...,negatif
3,salah satu produk lokal ter luv yg aku punya! ...,positif
4,Waw Kaka kulit nya bagus bgt pake sabun mandi ...,positif
5,Asalamualaikumyasalam emuah.cantiknya tasya fa...,positif


In [2]:
# Melihat jumlah data
df_comments['label'].value_counts()

label
negatif    3
positif    3
Name: count, dtype: int64

#### Preprocessing

In [3]:
import re
import string
from nltk.tokenize import word_tokenize
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from nltk.corpus import stopwords

stemmer = StemmerFactory().create_stemmer()
stopword_factory = StopWordRemoverFactory()
combined_stopwords = set(stopword_factory.get_stop_words()).union(set(stopwords.words('english')))

def clean_text(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    return text

def preprocess_text(text):
    text = clean_text(text).lower()
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in combined_stopwords]
    stemmed = [stemmer.stem(word) for word in tokens]
    return ' '.join(stemmed)


df_comments['preprocess'] = df_comments['comment'].apply(preprocess_text)


#### Transform TF data Test dengan data Train

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(df_comments['preprocess'])
y_train = df_comments['label']

terms = vectorizer.get_feature_names_out()
idf_values = vectorizer.idf_

print(y_train) # cetak label data train

0    negatif
1    negatif
2    negatif
3    positif
4    positif
5    positif
Name: label, dtype: object


In [5]:
from collections import Counter

# Function to compute raw TF (counts of terms in document)
def compute_raw_tf(doc):
    words = doc.split()
    count = Counter(words)
    return count

# Compute normalized TF
def compute_tf(doc):
    words = doc.split()
    count = Counter(words)
    total_terms = len(words)
    tf = {term: count[term] / total_terms for term in count}
    return tf

# Create DataFrame for TF-IDF
tfidf_df = pd.DataFrame(X_train.toarray().T, index=terms, columns=[f'D{i+1}' for i in range(len(df_comments['preprocess']))])
idf_df = pd.DataFrame(idf_values, index=terms, columns=["IDF"])

# Compute raw TF for each document (term count)
raw_tf_dicts = [compute_raw_tf(doc) for doc in df_comments['preprocess']]
raw_tf_df = pd.DataFrame(raw_tf_dicts, index=[f'D{i+1}' for i in range(len(df_comments['preprocess']))]).T


# Compute normalized TF for each document
tf_dicts = [compute_tf(doc) for doc in df_comments['preprocess']]
tf_df = pd.DataFrame(tf_dicts, index=[f'D{i+1}' for i in range(len(df_comments['preprocess']))]).T

# Fill NaN values with 0
raw_tf_df = raw_tf_df.fillna(0)
tf_df = tf_df.fillna(0)
idf_df = idf_df.fillna(0)
tfidf_df = tfidf_df.fillna(0)

# Sum all normalized TF values across all documents (TFNormAll)
tf_norm_all = tf_df.sum(axis=1)

# Compute Document Frequency (DF) - number of documents where the term appears
df_values = (raw_tf_df > 0).sum(axis=1)

# Create final DataFrame
final_df = pd.DataFrame(index=terms)
final_df['Terms'] = terms

# Add raw TF for each document (Raw Terms per document)
final_df = final_df.join(raw_tf_df.add_prefix('TF'))  # Add raw term counts for each document

# Add normalized TF for each document
final_df = final_df.join(tf_df.add_prefix('TFN'))  # Add normalized TF for each document

# Add sum of normalized TFs across all documents
final_df['TFNAll'] = tf_norm_all

# Add Document Frequency (DF) column
final_df['DF'] = df_values

# Add IDF column
final_df['IDF'] = idf_df['IDF']

# Add TF-IDF for each document
final_df = final_df.join(tfidf_df.add_prefix('TFIDF_'))

# Round all numeric columns to 3 decimal places
final_df = final_df.round(3)

# Export the final DataFrame to CSV
final_df.to_csv('train_metrics.csv', index=False)

#### Latih Model

In [6]:
from sklearn.svm import SVC
import joblib

result_path = result_path = os.getcwd()
model_file = os.path.join(result_path, 'trained_model.pkl')

if os.path.exists(model_file):
    model = joblib.load(model_file)
else:
    model = SVC(random_state=0, kernel='linear')
    model.fit(X_train, y_train)
    joblib.dump(model, model_file)


c:\000-Python-Project\ran-svm\notebook\trained_model.pkl


#### Prediksi Kelas Comment

In [11]:
new_comment = 'sudah banget nempel di muka' # text train
X_test = preprocess_text(new_comment)

X_test = vectorizer.transform([X_test])
predict = model.predict(X_test)
print(predict)

['negatif']
